# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import pickle

from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download(['punkt', 'wordnet'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
X = df['message']
Y = df.iloc[:, 3:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """
    Text tokenizer
    """
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vectorize', CountVectorizer(tokenizer=tokenize)),
        ('tf-idf', TfidfTransformer()),
        ('classifier', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_pred.shape

(6554, 36)

In [13]:
#y_pred = pipeline.predict(X_test)

for i in range(len(Y.columns)):
    print(Y.columns[i])
    print(classification_report(y_test.iloc[:, i], y_pred[:, i])

                        precision    recall  f1-score   support

               related       0.61      0.36      0.45      1513
               request       0.82      0.93      0.87      4993
                 offer       0.56      0.10      0.18        48

           avg / total       0.77      0.79      0.77      6554



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))


In [14]:
print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

             precision    recall  f1-score   support

          0       0.61      0.36      0.45      1513
          1       0.82      0.93      0.87      4993
          2       0.56      0.10      0.18        48

avg / total       0.77      0.79      0.77      6554



### 6. Improve your model
Use grid search to find better parameters. 

In [30]:
parameters = {
    'classifier__estimator__n_estimators': [100, 200],
    'classifier__estimator__criterion': ['gini', 'entropy'],
    'classifier__estimator__max_depth': [6, 18]
}

cv = GridSearchCV(pipeline, parameters, n_jobs=-1).fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [31]:
y_pred = cv.predict(X_test)

i = 0
print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1580
          1       0.75      1.00      0.86      4934
          2       0.00      0.00      0.00        40

avg / total       0.57      0.75      0.65      6554



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
import pickle

pickle_file = open('model.pkl', 'wb')
pickle.dump(cv, pickle_file)
pickle_file.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.